In [1]:
# import sklearn
import pandas as pd
from sklearn import tree
from sklearn import datasets
from sklearn.model_selection import train_test_split
import graphviz 

In [226]:
iris = datasets.load_iris()

def train(feature, label, feature_names, cls='entropy'):
    x_train, x_test, y_train, y_test = train_test_split(feature, label, test_size=0.33, shuffle=True)

    clf = tree.DecisionTreeClassifier(criterion=cls)
    clf.fit(x_train, y_train)

    s = clf.score(x_test, y_test)
    print(s)

    with open("out.dot", 'w') as f :
        f = tree.export_graphviz(clf, out_file=f,
                feature_names=feature_names)

    dotdata = tree.export_graphviz(clf, out_file=f,
                feature_names=feature_names)

    import os  
    os.environ['PATH'] = os.pathsep + r'D:\tools\code\Graphviz\bin'
    os.system('dot -Tpng out.dot -o 决策树模型.png')

In [53]:
data = pd.read_excel('data/第1阶段.xlsx', index_col=None)
print(data.dtypes)


q1a              object
q1b               int64
rpa101          float64
rpa102          float64
rpa103          float64
                 ...   
fs1ct7y2        float64
Unnamed: 219    float64
ID               object
center            int64
WOMAC           float64
Length: 223, dtype: object


In [135]:
data = pd.read_excel('data/第1阶段.xlsx', index_col=None)
print(data.shape)

data['WOMAC'].fillna(data['WOMAC'].mean(), inplace=True)

data_type = data.dtypes
drop_string_index = data_type[data_type==object].index.values
print(drop_string_index)
data.drop(drop_string_index, axis=1, inplace=True)

def have_null(data):
    df = data.isnull()
    for u in df.columns:
        if df[u].dtype==bool:
            df[u]=df[u].astype('int')
    return {
        "col_sum": df.sum(axis=0), 
        "row_sum": df.sum(axis=1),
        "index": df.columns}
print(data.shape)

(1951, 223)
['q1a' 'fs1q1' 'fs1q6q' 'fs1q7q' 'fs1b23' 'fs1c1c1q' 'fs1c1c2q' 'fs1c2x'
 'fs1e3q' 'fs1e6q' 'fs1e9q' 'fs1e10q' 'ID']
(1951, 210)


In [181]:
import copy

data_copy = copy.deepcopy(data)
womac = copy.deepcopy(data_copy['WOMAC'].values)
count = len(womac)

womac.sort()
range1 = womac[int(count/4)]
range2 = womac[int(count*3/4)]

print(data_copy.shape)

print(f"womac splite range: {range1} {range2}")
def get_cls(x):
    if x < range1:
        return 1
    elif x > range2:
        return 3
    else:
        return 2

data_copy['WOMAC'] =  data_copy['WOMAC'].apply(get_cls)

print(data_copy.shape)
# print(data_copy)

(1951, 210)
womac splite range: 32.0 48.0
(1951, 210)


In [206]:
# step1: fillnan  
col_nan_num = 210
row_nan_num = 30

# 列级初步处理
col_analise, row_analise, all_col_index = have_null(data_copy).values()
# print(col_analise)
drop_col_index = col_analise[col_analise>col_nan_num].index
print(f"null more than {col_nan_num}'s cols will be delete")
print(f"drop col count: {len(drop_col_index)}")
print(f"drop col name: {drop_col_index.values}")

data_drop_col = data_copy.drop(drop_col_index, axis=1, inplace=False)

# 行级初步处理
col_analise, row_analise, all_col_index = have_null(data_drop_col).values()
# print(row_analise)
drop_row_index = row_analise[row_analise>row_nan_num].index
print(f"null more than {row_nan_num}'s rows will be delete")
print(f"drop row count: {len(drop_row_index)}")
print(f"drop row number: {drop_row_index.values}")


data_drop_row = data_drop_col.drop(drop_row_index, axis=0, inplace=False)

print(f"now data shape: {data_drop_row.shape}")
print(f"now col name: {data_drop_row.columns.values}")

# 按特征含义处理剩余缺省值
# TODO 确认每一列的具体含义，制定不同的处理方案
data_drop_fillna = data_drop_row.dropna()
print(f"now data shape: {data_drop_fillna.shape}")


# print(data_drop_row)
feature = data_drop_fillna.drop('WOMAC', axis=1, inplace=False)
label = data_drop_fillna['WOMAC'].values
label_name = ['front', 'middle', 'end']
feature_name = feature.columns
print(feature_name)


null more than 210's cols will be delete
drop col count: 54
drop col name: ['rpa101' 'rpa102' 'rpa103' 'rpa104' 'rpa105' 'rpa106' 'rpa107' 'rpa2'
 'rpb1' 'rpb2' 'rpb3' 'rpb4' 'rpb5' 'rpb6' 'rpc' 'rpd' 'fs1q7' 'fs1c1a1'
 'fs1c1a2' 'fs1c1b' 'fs1c1c1' 'fs1c1c2' 'fs1c1e1' 'fs1c1e2' 'fs1c1e3'
 'fs1c1e4' 'fs1c2c' 'fs1c301q' 'fs1c302q' 'fs1c303q' 'fs1c304q' 'fs1c305q'
 'fs1c306q' 'fs1c307q' 'fs1c308q' 'fs1c309q' 'fs1c4a' 'fs1c4b' 'fs1c4c'
 'fs1c6a1a' 'fs1c6a1b' 'fs1c6a2a' 'fs1c6a2b' 'fs1c6b1a' 'fs1c6b1b'
 'fs1c6b2a' 'fs1c6b2b' 'fs1c6c1a' 'fs1c6c1b' 'fs1c6c2a' 'fs1c6c2b'
 'fs1d2a' 'fs1e6' 'Unnamed: 219']
null more than 30's rows will be delete
drop row count: 44
drop row number: [  22   35   40   54   79   82  103  116  121  122  124  126  133  135
  140  146  149  161  166  169  207  212  236  242  244  278  284  291
  315  361  364  369  483  562 1051 1058 1523 1529 1579 1598 1661 1857
 1911 1912]
now data shape: (1907, 156)
now col name: ['q1b' 'fs1q2' 'age' 'fs1q3a' 'fs1q3b' 'fs1q3c' 'fs1q

In [266]:
# classification
# train(feature, label, feature_name, cls="gini")
train(feature, label, feature_name, cls="entropy")

0.8158844765342961
